## Imports

In [2]:
!pip install pyccel

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import njit
from pyccel.epyccel import epyccel
from pyccel.decorators import types

KeyError: 'MSMPI_INC'

## Pure python function

In [ ]:
def solve_1d_linearconv_pure(u, un, nt, nx, dt, dx, c):

    for n in range(nt):  
        for i in range(nx): un[i] = u[i]
        for i in range(1, nx): 
            u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])
    return 0

## Numba accelerator

In [ ]:
@njit(fastmath=True)
def solve_1d_linearconv_numba(u, un, nt, nx, dt, dx, c):

    for n in range(nt):  
        for i in range(nx): un[i] = u[i]
        for i in range(1, nx): 
            u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])
    return 0

## Pyccel accelerator

In [ ]:
def solve_1d_linearconv_pyccel(u:'float[:]', un:'float[:]', nt:'int', nx:'int', dt:'float', dx:'float', c:'float'):

    for n in range(nt):  
        for i in range(nx): un[i] = u[i]
        for i in range(1, nx): 
            u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])
    return 0

## Inputs

In [ ]:
nx = 201  # try changing this number from 41 to 81 and Run All ... what happens?
dx = 2 / (nx-1)
nt = 200    #nt is the number of timesteps we want to calculate
c = 1.      #assume wavespeed of c = 1
dt = 0.004
grid = np.linspace(0,2,nx) # creating the space grid    

u0 = np.ones(nx)      #numpy function ones()
u0[int(.5 / dx):int(1 / dx + 1)] = 2  #setting u = 2 between 0.5 and 1 as per our I.C.s
plt.plot(grid,u0)
u = u0.copy()
un = np.ones(nx)

solve_1d_linearconv_pure(u, un, nt, nx, dt, dx, c)
#Plotting data
plt.plot(grid,u);
plt.ylim([1.,2.2])
plt.xlabel('X')
plt.ylabel('Velocity')    
plt.title('1D Linear Convection')

## Call Funtions

### Pure function

In [ ]:
%timeit solve_1d_linearconv_pure(u, un, nt, nx, dt, dx, c)

### Function with numba accelerator

In [ ]:
%timeit solve_1d_linearconv_numba(u, un, nt, nx, dt, dx, c)

### Function with pyccel accelerator

In [ ]:
solve_1d_linearconv_f90 = epyccel(solve_1d_linearconv_pyccel)

In [ ]:
%timeit solve_1d_linearconv_f90(u, un, nt, nx, dt, dx, c)